Title   : Script for combining raw SLS .txt files, plotting columns and adding them to an exported file with sheets combining all raw data

Author  : Zaf Khalil

Date    : 28-June-2024


# Libraries
Load this first to make sure the libraries are imported

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.chart import LineChart, Reference
from openpyxl.chart.axis import ChartLines
import io
import os

# Combining .txt raw files in one excel sheet
This section will import the raw data in a specified path and will combine all found files in one output xlsx file
The files have to be the raw .txt files from the SLS export options.

1- Put the script file in the folder that contains a subfolder of the raw data

2- Insert the name of the raw data subfolder as a raw_directory

3- Choose a name for the file containing combined raw data as sheets

4- Run the cell

The output will be found in the main folder in which the script is placed.

In [21]:
# Define the directory and output filename
raw_directory = 'rawdata'                                                                          # Raw data folder name in the directory
combined_sheets_filename = 'combined sheets.xlsx'                                                   # Name of the file in which all sheets are to be combined
dfs = {}                                                                                        # Creating an empty dataframe

# Iterate over each file in the directory
for filename in os.listdir(raw_directory):                                                      
    if filename.endswith(".txt"):                                                               # Condition to fine .txt files in the directory
        file_path = os.path.join(raw_directory, filename)                                       # Creating a temp datafile name by combining the found file name and the directory to be read

        # Read the main data, skipping the first 41 rows
        main_data = pd.read_csv(                                                                
            file_path,                                                                          # Opens the current file in the for-loop
            skiprows=41,                                                                        # skips 41 rows, and reads the remainder of the data
            sep='\t', encoding='latin1',
            skipfooter=1,
            engine='python'
        )
        
        # Read the first 17 rows of the original file
        first_17_rows = pd.read_csv(                                                            
            file_path, nrows=16,                                                                # Opens the current file and extracts 16 rows
            skiprows=lambda x: x==8,                                                            # Skips only the 9th row
            sep='\t', encoding='latin1',
            header=None                                                                         # Does not extract a header
        )
        # Read the transmittance rows in the raw data files
        transmittance_rows = pd.read_csv(                                                                   
            file_path, nrows=2,                                                                 # Opens the current file again and extract only 2 row
            skiprows=18,                                                                        # Skips 18 rows and extracts the 19th and 20th rows based on the previous line
            sep='\t', encoding='latin1',
            header=None
        )

        # Read the data ID row in the raw data files
        ID_row = pd.read_csv(                                                                   
            file_path, nrows=1,                                                                 # Opens the current file again and extract only 1 row
            skiprows=37,                                                                        # Skips 37 rows and extracts the 38th row based on the previous line
            sep='\t', encoding='latin1',
            header=None
        )

        # Read the reflextive index rows in the raw data files
        reflect_sample_info_rows = pd.read_csv(                                                                   
            file_path, nrows=7,                                                                 # Opens the current file again and extract 7 rows
            skiprows=25,                                                                        # Skips 25 rows and extracts the rows based on the previous line
            sep='\t', encoding='latin1',
            header=None
        )

        empty_row = pd.Series([None])                                                           # Creates an empty row to add to the next data frame (avoids having one of the values being added as a header by concat)
        first_17_rows = pd.concat(
            [empty_row,
             ID_row,
             first_17_rows,
             transmittance_rows,
             reflect_sample_info_rows
            ],
            ignore_index=True
        )                                                                                       # Combines the extracted rows above in one data frame

        # Add the first 17 rows as new columns to the main DataFrame
        main_data2 = pd.concat([main_data, first_17_rows], axis=1)                              

        # Store the DataFrame in the dictionary
        dfs[filename] = main_data2

# Write all DataFrames to an Excel file with each DataFrame in a separate sheet                 
with pd.ExcelWriter(combined_sheets_filename, engine='openpyxl') as writer:
    for sheet_name, df in dfs.items():
        # Use the file name without extension as the sheet name
        df.to_excel(writer, index=False, sheet_name=os.path.splitext(sheet_name[:31])[0])       # Sheet_name[-31:] is used to rename the new sheets using the last 31 characters of the original sheets. If the first 31 characters are identical, excel will overwrite sheets and not all data will be outputted.

print(f"Data has been written to {combined_sheets_filename}")                                   

Data has been written to combined sheets.xlsx


# This part saves the plots as adjustable excel charts

In [22]:
# Step 1: Import the Excel file with multiple sheets
file_path = combined_sheets_filename                                                        # Replace with your Excel file path --> the variable here matches the name of the exported combined sheets after importation above.
xls = pd.ExcelFile(file_path)                                                               # Opens/reads the target excel file (with combined raw data values)

# Step 2: Create the output subfolder
output_file_name = 'output.xlsx'                                                           # Choose a name for the output file (data, sample name, user name, etc)
output_folder = 'Output'                                                            # Choose a name for the output folder to be created
if not os.path.exists(output_folder):                                                       # Checks if the name does not already excists, if it exists, it will use it, if not, it will create a new folder
    os.makedirs(output_folder)

# Step 3: Initialize a writer for the output Excel file in the output subfolder
# This creates a variable in which a funciton is used to combine a directory and an output file name. Basically creating a name for the output file in the output folder.
output_file = os.path.join(output_folder, output_file_name)                                 

#The writer function is defined and the engine is xlswriter. This will 'write' the final results in an output file.
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')   

# Step 4: Iterate through each sheet, create scatter plots with straight lines, and save them on each sheet
for sheet_name in xls.sheet_names:
    # Truncate the sheet name to 31 characters if necessary
    # (this is because writer encouters issues when trying to name sheets with more than 31 characters)
    truncated_sheet_name = sheet_name[:31]

    # Read each sheet into a DataFrame
    df2 = pd.read_excel(xls, sheet_name=sheet_name)

    # Write the DataFrame to the Excel writer
    # Here you see that the written name of the sheets in the new output file is the one with a maximum of 31 characters
    df2.to_excel(writer, sheet_name=truncated_sheet_name, index=False)                      

    # Get the xlsxwriter workbook and worksheet objects. (Basically, opening a new file to write the save the new output in it)
    workbook = writer.book
    worksheet = writer.sheets[truncated_sheet_name]

    # Create a new scatter chart object
    chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight'})                  # Scatter plot with straight line is needed in order to have a value-based x-axis. Otherwise, logarithmic view of x-axis is not possible

    # Since we plot 2 lines and use 2 y-axes, we will have to define to series and each will be assigned a different y-axis and have different properties.
    # Configure the first series
    chart.add_series({
        'name':       df2.columns[1],                                                       # The title of the first series data
        'categories': [truncated_sheet_name, 1, 0, len(df2), 0],                            # defining the dimensions of the data (sheetname, first_row, first_col, last_row (len gives the total length, used as last row number), last_col (we are plotting one column here to first_col and last_col are the same))
        'values':     [truncated_sheet_name, 1, 1, len(df2), 1],                            # Adjust based on your data range (Choosing the data range to be plotted)
        'y2_axis':    False,                                                                # Here we switch off the second y-axis for the first series and have the main y-axis on by default.
        'marker':     {'type': 'none'},                                                     # Hide markers
        'line':       {'color': 'black', 'width': 1},                                       # Set the color of the line
    })

    # Configure the second series for the secondary y-axis
    chart.add_series({
        'name':       df2.columns[2],                                                       # The title of the first series data
        'categories': [truncated_sheet_name, 1, 0, len(df2), 0],                            # defining the dimensions of the data
        'values':     [truncated_sheet_name, 1, 2, len(df2), 2],                            # Adjust based on your data range
        'y2_axis':    True,                                                                 # Here we switch on the second y-axis for the second series.
        'marker':     {'type': 'none'},                                                     # Hide markers
        'line':       {'color': 'blue', 'dash_type': 'dash', 'width': 1},                   # Set the color of the line
    })

    # Add a chart title and some axis labels, configuration of the main chart
    chart.set_title({'name': f'{truncated_sheet_name} Plot',                                # The title of the entire chart/plot
                     'name_font': {'size':12}
                    })                               
    
    # Configuration of the x-axis of the main plot
    chart.set_x_axis({                                                                      # Properties of the x-axis
        'name': df2.columns[0],                                                             # title of the x-axis
        'min': 0.1, 'max':5000,                                                             # range of the x-axis
        'log_base': 10,                                                                     # logarithmic scaling
        'crossing':0.1,                                                                     # defining location of the y-axis
        'major_gridlines': {'visible': True, 'line': {'color': 'gray', 'width': 0.5}},      # adding major gridlines
        'minor_gridlines': {'visible': True, 'line': {'color': '#DCD8D8', 'width': 0.5}}    # adding minor gridlines
    })
    # Configuration of the first y-axis
    chart.set_y_axis({                                                                      
        #'name': df2.columns[1],                                                            # The title of the axis. Can either match the original header or add the title you want after the : in ''
        'max': 20,                                                                          # Setting maximum value of the axis
        'major_gridlines':{'visible':False}                                                 # Hiding the horizontal gridlines
    })
    # Configuration of the second y-axis
    chart.set_y2_axis({
        #'name': df2.columns[2],                                                            # This makes the title matches the original title in the header of the raw data
        'name': 'Cumulative (%)',                                                           # This is a user chosen title
        'max': 100,                                                                         # Setting maximum value of the axis
        'major_gridlines':{'visible':False}                                                 # Hiding the horizontal gridlines of the second y-axis
    })

    # Set the size of the chart
    chart.set_size({'width': 400, 'height': 350})                                           # Set chart size (width x height)

    # Set the legend position to the bottom
    chart.set_legend({'position': 'bottom'})

    # Insert the chart into the worksheet (in output file)
    worksheet.insert_chart('I4', chart)

# Step 5: Save the Excel file with inserted charts as "[output file name].xlsx"
writer.close()

print(f"Plots saved and Excel file '{output_file}' created.")

Plots saved and Excel file 'Output\output.xlsx' created.


# This part saves the plots as .png images in the combined excel file

In [ ]:
# Step 1: Import the Excel file with multiple sheets
file_path = combined_sheets_filename                                                            # This will automatically use the name you specified as a combined sheets output file
xls = pd.ExcelFile(file_path)

# Step 2: Create the output subfolder
output_folder = 'Output(PNG plots)'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Step 3: Initialize a writer for the output Excel file in the output subfolder
output_file = os.path.join(output_folder, 'output.xlsx')
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

# Step 4: Iterate through each sheet, create line plots, and save them on each sheet
for sheet_name in xls.sheet_names:
    # Truncate the sheet name to 31 characters if necessary
    truncated_sheet_name = sheet_name[:31]

    # Read each sheet into a DataFrame
    df2 = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Create a line plot for the first 2 columns
    f, ax = plt.subplots()
    ax.plot(df2.iloc[:, 0], df2.iloc[:, 1], c="C1")                                             # Assuming the first two columns are to be plotted
    ax.set_ylabel(df2.columns[1], c="C0")                                                       # Title of the y-axis maching the header of the second column
    ax.set_ylim(0, 20)                                                                          # Setting the range of the y-axis
    ax2 = ax.twinx()                                                                            # Creating a second plot to be merged
    ax2.plot(df2.iloc[:, 0], df2.iloc[:, 2])                                                    # Plotting the first and third columns against each other
    ax2.set_ylabel(df2.columns[2], c='C1')                                                      # Setting the title of the y axis of the xecond plot
    ax2.set_ylim(0, 100)                                                                        # Setting the range of the second y-axis
    ax.grid()                                                                                   # Switching on gridlines
    ax.set_xscale("log")                                                                        # Changing the x-axis to logarithmic scaling
    
    # Save the plot as a temporary image file in the output subfolder
    plot_filename = os.path.join(output_folder, f"{truncated_sheet_name}_plot.png")
    f.savefig(plot_filename)
    plt.close(f)                                                                                # This closes the plots after creating them in the for-loop. Add # before it to see the plots below
    
    # Write the DataFrame to the Excel writer
    df2.to_excel(writer, sheet_name=truncated_sheet_name, index=False)                          
    
    # Insert the plot image into the Excel sheet
    workbook = writer.book                                                                      
    worksheet = writer.sheets[truncated_sheet_name]
    worksheet.insert_image('G1', plot_filename)                                                 # Here you can choose the location of the plot in the sheets "G1" is cell G1 in an excel sheet

# Step 5: Save the Excel file with inserted plots as "output.xlsx"
writer.close()

print(f"Plots saved and Excel file '{output_file}' created.")